## Overview

The "MLflow PySpark Pipeline for Diabetes Prediction" - Python Notebook is a comprehensive example of how to use the MLflow library to build a machine learning pipeline to predict diabetes in patients. The notebook demonstrates how to use various PySpark libraries, such as Pipeline, PipelineModel, Logistic Regression, BinaryClassificationEvaluator, and MulticlassClassificationEvaluator to build and evaluate the machine learning model.

**This Python Notebook runs only in Azure/GCP/AWS Databricks.**

The first step is to set the MLflow experiment for diabetes prediction. The next step is to load the diabetes dataset, which is split into training and test sets using the randomSplit function. The hyperparameters for the logistic regression model are set, and the model is trained using the pipeline function.

The categorical features in the dataset are converted to numeric using StringIndexer, and the features vector is created using VectorAssembler. The logistic regression model is defined, and the pipeline is created using these components. The hyperparameters are logged, and the model is trained.

After training the model, the predictions are made on the test data, and the model is evaluated using BinaryClassificationEvaluator and MulticlassClassificationEvaluator. The evaluation metrics, such as area under the ROC curve and accuracy, are logged using the mlflow.log_metric function.

The model is then logged using the mlflow.spark.log_model function, which saves the model to the MLflow registry. Finally, the production version of the model is transitioned using the MlflowClient function. Then, The model is then loaded and used to make predictions on new data.

This notebook is written in **Python** so the default cell type is Python.

In [ ]:
import mlflow
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

# Set the MLflow experiment
mlflow.set_experiment("diabetes-prediction")


# Load the diabetes data
sc = SparkContext('local')
spark = SparkSession(sc)
data = spark.read.csv("./diabetes_prediction_dataset.csv", inferSchema=True,header=True)

# Convert the diabetes label to double
data = data.withColumn("diabetes", col("diabetes").cast(DoubleType()))

# Split the data into training and test sets
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=1234)

# Set the hyperparameters for the logistic regression model
lr_params = {
    "regParam": 0.01,
    "elasticNetParam": 0.0,
    "maxIter": 100
}


# Train the model and make predictions on the test data
with mlflow.start_run():

    # Convert categorical features to numeric using StringIndexer
    GenderIndexer = StringIndexer(inputCol="gender", outputCol="genderIndex")                       
    SmokeHistIndexer = StringIndexer(inputCol="smoking_history", outputCol="smoking_statusIndex")  

    # Create the features vector using VectorAssembler
    F_assembler = VectorAssembler(inputCols=["genderIndex", "age", "hypertension", "heart_disease", "smoking_statusIndex", "bmi","HbA1c_level","blood_glucose_level"], outputCol="features")
    
    # Define the logistic regression model
    lr = LogisticRegression(featuresCol="features", labelCol="diabetes")
  
    # Define the pipeline
    pipeline = Pipeline(stages=[GenderIndexer, SmokeHistIndexer,F_assembler, lr])
    
    # Log the hyperparameters
    mlflow.log_params(lr_params)
    
    # Train the model
    models=pipeline.fit(trainingData)
    
    # Make predictions on the test data
    predictions = models.transform(testData)
    
    # Evaluate the model
    auc_evaluator = BinaryClassificationEvaluator(labelCol="diabetes", rawPredictionCol="prediction", metricName="areaUnderROC")
    areaUnderROC = auc_evaluator.evaluate(predictions)
    print("Area under ROC curve: {}".format(areaUnderROC))
    acc_evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="diabetes", metricName="accuracy")
    accuracy = acc_evaluator.evaluate(predictions)
    print("Accuracy: {:.2f}%".format(accuracy*100))
    
    #Log the metrics
    mlflow.log_metric("Accuracy", accuracy*100)
    mlflow.log_metric("AUC", areaUnderROC)
    
    # Log the model
    mlflow.spark.log_model(models,"model")

    #Get the ID of the current run
    runid=mlflow.active_run().info.run_id
    model_uri='runs:/'+runid+'/model' #Here we are constructing the model URI using the run ID of the active run and the 'model' endpoint
    model_name="Diabetes_prediction_model" #Set the name for the registered model
    model_details=mlflow.register_model(model_uri=model_uri,name=model_name) #Register the trained model with the name and URI specified

2023/09/25 19:30:58 INFO mlflow.tracking.fluent: Experiment with name 'diabetes-prediction' does not exist. Creating a new experiment.
23/09/25 19:31:00 WARN Utils: Your hostname, JamesoudeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.71 instead (on interface en0)
23/09/25 19:31:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 19:31:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/25 19:31:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/09/25 19:31:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [ ]:
from mlflow.tracking.client import MlflowClient  #Import the Mlflow client library

client=MlflowClient() #Create a client instance for accessing the Mlflow tracking server
client.transition_model_version_stage(name=model_details.name,version=model_details.version,stage='Production') #Staging the registered model version to the production stage

In [ ]:
import mlflow
#Load the saved MLflow model
loaded_model = mlflow.spark.load_model(model_uri)

#Create a new test dataset as a Spark dataframe
test = spark.createDataFrame([
    ('Female',39.0,0,1,"No Info",79,8.8,145)], ["gender", "age", "hypertension","heart_disease","smoking_history","bmi","HbA1c_level","blood_glucose_level"])

#Use the loaded model to make predictions on the new dataset
new_predictions=loaded_model.transform(test)
#Display the predicted output
new_predictions.show()

In [ ]:
import pandas as pd
dict_data = {'gender': 'Female', 'age': 39, 'hypertension': 1, 'heart_disease': 0, 'smoking_history': 'No Info', 'bmi': 79, 'HbA1c_level': 8.8, 'blood_glucose_level': 145}
df = pd.DataFrame.from_dict(dict_data, orient="index")
loaded_model = mlflow.pyfunc.load_model(model_uri)
y_pred = loaded_model.predict(df)
print(y_pred)